In [86]:
from datetime import date, time, datetime
from google.oauth2 import service_account
from google.cloud import bigquery, storage
import pandas as pd
import json

In [87]:
dev = True
if (dev):
    key_path = r"C:\Users\nites\OneDrive\Documents\napo-nitesh-local-ae32-vpcservice-datawarehouse-879a160a28ee.json"
    credentials = service_account.Credentials.from_service_account_file(
        key_path, scopes=["https://www.googleapis.com/auth/cloud-platform"],
    )

    client = bigquery.Client(credentials=credentials, project=credentials.project_id,)
    storage_client = storage.Client(credentials=credentials, project=credentials.project_id,) 

else: 
    client = bigquery.Client()
    storage_client = storage.Client()

In [88]:
job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.NEWLINE_DELIMITED_JSON,
    autodetect=True,
    max_bad_records=1,
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
)

In [89]:
bucket_name='data-warehouse-harbour'
dataset_name='postgres'
path_name='policy-service/{}'.format(datetime.now().strftime("%Y-%m-%d"))
bucket = storage_client.get_bucket(bucket_name)

In [90]:
table_mapping = {
    "policy.breed.json":"breed",
   # "policy.customer.json":"customer",
    "policy.pet.json":"pet",
   # "policy.policy.json":"policy",
    "policy.product.json":"product",
    "policy.productline.json":"productline",
    "policy.payment.json":"policy_payment"
}

In [12]:
#print(table_mapping,table_id,uri,table_mapping["policy.customer.json"])

In [91]:
for table in table_mapping:
    uri = "gs://{}/{}/{}".format(bucket_name,path_name,table)
    #size_in_bytes = storage_client.bucket.get_blob(uri).size
    #print(table,size_in_bytes)
    
    if (storage_client.get_bucket(bucket_name).get_blob(path_name+'/'+table).exists()):
        size=storage_client.get_bucket(bucket_name).get_blob(path_name+'/'+table).size
        print(table,' was found in today\'s bucket with file size',size)
        if (size==0):
            raise NameError(table+' file size is 0')

        table_id = "ae32-vpcservice-datawarehouse.{}.{}".format(dataset_name,table_mapping[table])
        load_job = client.load_table_from_uri(
            uri,
            table_id,
            location="EU",  # To match the destination dataset location
            job_config=job_config,
            
        )
        print(table,load_job.result())


    else:
        print(table,' not found in today\'s bucket')
        raise NameError(table+' not found in bucket')

policy.breed.json  was found in today's bucket with file size 346042
policy.breed.json LoadJob<project=ae32-vpcservice-datawarehouse, location=EU, id=ebe25be3-8fcd-4bea-aa6e-cbc4c4d0f37f>
policy.pet.json  was found in today's bucket with file size 30134034
policy.pet.json LoadJob<project=ae32-vpcservice-datawarehouse, location=EU, id=01bfd90c-f487-439d-acd8-d034086d5d64>
policy.product.json  was found in today's bucket with file size 7219
policy.product.json LoadJob<project=ae32-vpcservice-datawarehouse, location=EU, id=9a0ef1f6-fd1c-4cd9-bbac-be8d39391321>
policy.productline.json  was found in today's bucket with file size 304
policy.productline.json LoadJob<project=ae32-vpcservice-datawarehouse, location=EU, id=fd3d2481-d8a3-47b4-92ef-3f2e8ece12d8>
policy.payment.json  was found in today's bucket with file size 83874942
policy.payment.json LoadJob<project=ae32-vpcservice-datawarehouse, location=EU, id=43996dfc-cf17-4e72-ba97-769500690a6e>


In [94]:
# Subscription table loading
blob = bucket.blob("{}/{}".format(path_name,'policy.subscription.json'))
df = pd.read_json(str(blob.download_as_bytes(),'utf-8'), lines=True)
df1 = pd.json_normalize(df['fields'])

In [95]:
if 'policyid' in df1.columns:
    policy_column = 'policyid' 
if '_policyid' in df1.columns:
    policy_column = '_policyid'
    df1.rename(columns={'_policyid':'policyid'},inplace=True)

In [96]:
if (df1.policyid.count()==df.model.count()):
    print('True')
    df_final = pd.merge(df, df1, left_index=True, right_index=True)
    df_final.drop(columns=['fields','failed_payment_events'],inplace=True)

job_config2 = bigquery.LoadJobConfig(
    autodetect=True,
    write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
)

load_job = client.load_table_from_dataframe(
    df_final
    ,"ae32-vpcservice-datawarehouse.{}.subscription".format(dataset_name)
    ,job_config=job_config2
    ,location='EU'
) 
print(load_job.result())

True
LoadJob<project=ae32-vpcservice-datawarehouse, location=EU, id=89b1be92-97e2-4164-a9a2-3b42b452585b>
